In [ ]:
pip install nltk

In [141]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [142]:
# Note: Change the RAW URL before running the model
df = pd.read_csv('C:/Users/maddy/OneDrive/Desktop/DATASET/Features_For_Traditional_ML_Techniques.csv')
df.head(5)

,Unnamed: 0,majority_target,statement,BinaryNumTarget,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,determiners,conjunctions,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq
0,0,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,...,0,0,5,0,1,0,33,3,5,19
1,1,True,End of eviction moratorium means millions of A...,1.0,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,...,0,2,1,0,0,0,14,0,2,34
2,2,True,End of eviction moratorium means millions of A...,1.0,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,...,0,1,0,0,0,0,3,0,4,10
3,3,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders\n\nBroken campaign promi...,4262.0,3619.0,34945.0,16423.0,44.0,...,0,1,3,0,0,1,6,8,1,30
4,4,True,End of eviction moratorium means millions of A...,1.0,@OhComfy I agree. The confluence of events rig...,70.0,166.0,15282.0,2194.0,0.0,...,0,1,3,0,1,0,11,3,2,19


In [143]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk

In [144]:
example = df['statement']
print(example)

0         End of eviction moratorium means millions of A...
1         End of eviction moratorium means millions of A...
2         End of eviction moratorium means millions of A...
3         End of eviction moratorium means millions of A...
4         End of eviction moratorium means millions of A...
                                ...                        
134193    Joe Bidens great-grandfather Joseph J. Biden w...
134194    Joe Bidens great-grandfather Joseph J. Biden w...
134195    Joe Bidens great-grandfather Joseph J. Biden w...
134196    Joe Bidens great-grandfather Joseph J. Biden w...
134197    Joe Bidens great-grandfather Joseph J. Biden w...
Name: statement, Length: 134198, dtype: object


In [145]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [146]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove non-alphanumeric characters
    text = re.sub(r'\W+', ' ', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    # Join tokens back into a single string
    preprocessed_text = ' '.join(lemmatized_tokens)
    
    return preprocessed_text

In [147]:
df['preprocessed_text'] = df['statement'].apply(preprocess_text)

In [148]:
df['preprocessed_text']

0         end eviction moratorium mean million american ...
1         end eviction moratorium mean million american ...
2         end eviction moratorium mean million american ...
3         end eviction moratorium mean million american ...
4         end eviction moratorium mean million american ...
                                ...                        
134193    joe bidens great grandfather joseph j biden sl...
134194    joe bidens great grandfather joseph j biden sl...
134195    joe bidens great grandfather joseph j biden sl...
134196    joe bidens great grandfather joseph j biden sl...
134197    joe bidens great grandfather joseph j biden sl...
Name: preprocessed_text, Length: 134198, dtype: object

In [149]:
from nltk.sentiment import SentimentIntensityAnalyzer


In [150]:
from textblob import TextBlob


In [151]:
df['sentiment_score'] = df['preprocessed_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [152]:
df['sentiment_label'] = df['sentiment_score'].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral')

In [153]:
df

,Unnamed: 0,majority_target,statement,BinaryNumTarget,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq,preprocessed_text,sentiment_score,sentiment_label
0,0,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,...,0,1,0,33,3,5,19,end eviction moratorium mean million american ...,-0.104167,Negative
1,1,True,End of eviction moratorium means millions of A...,1.0,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,...,0,0,0,14,0,2,34,end eviction moratorium mean million american ...,-0.104167,Negative
2,2,True,End of eviction moratorium means millions of A...,1.0,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,...,0,0,0,3,0,4,10,end eviction moratorium mean million american ...,-0.104167,Negative
3,3,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders\n\nBroken campaign promi...,4262.0,3619.0,34945.0,16423.0,44.0,...,0,0,1,6,8,1,30,end eviction moratorium mean million american ...,-0.104167,Negative
4,4,True,End of eviction moratorium means millions of A...,1.0,@OhComfy I agree. The confluence of events rig...,70.0,166.0,15282.0,2194.0,0.0,...,0,1,0,11,3,2,19,end eviction moratorium mean million american ...,-0.104167,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134193,134193,False,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,Joe Biden's family owned African slaves....\n\...,2075.0,2662.0,156306.0,62375.0,3.0,...,0,3,0,31,0,4,32,joe bidens great grandfather joseph j biden sl...,0.800000,Positive
134194,134194,False,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,"Joe Bidens great, great grandfather was a slav...",1636.0,1626.0,215290.0,44011.0,10.0,...,0,0,0,16,6,2,8,joe bidens great grandfather joseph j biden sl...,0.800000,Positive
134195,134195,False,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,"@ChevyChaseToGo ""Joe Bidens great-grandfather ...",286.0,2501.0,28377.0,7617.0,1.0,...,0,0,1,2,4,0,37,joe bidens great grandfather joseph j biden sl...,0.800000,Positive
134196,134196,False,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,@JoeBiden Facts are Bidens VP Kamala Harris Gr...,2.0,45.0,16.0,509.0,0.0,...,0,0,0,0,0,1,11,joe bidens great grandfather joseph j biden sl...,0.800000,Positive


In [155]:
df['sentiment_label'].unique()

array(['Negative', 'Positive', 'Neutral'], dtype=object)

In [156]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Bidirectional, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.regularizers import l2

In [157]:
X = df[['statement', 'sentiment_score']]
y = df['BinaryNumTarget']

In [158]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [159]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['statement'])

In [160]:
X_train_seq = tokenizer.texts_to_sequences(X_train['statement'])
X_test_seq = tokenizer.texts_to_sequences(X_test['statement'])

In [161]:
max_seq_length = max(len(seq) for seq in X_train_seq)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_seq_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_seq_length)

In [162]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_seq_length))
model.add(Bidirectional(LSTM(units=64, kernel_regularizer=l2(0.1), recurrent_regularizer=l2(0.1))))
model.add(Dense(1, activation='sigmoid'))

In [163]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 49, 128)           490880    
                                                                 
 bidirectional_21 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dense_21 (Dense)            (None, 1)                 129       
                                                                 
Total params: 589,825
Trainable params: 589,825
Non-trainable params: 0
_________________________________________________________________


In [166]:
model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=10, batch_size=32)


Epoch 1/10
3355/3355 [==============================] - 374s 110ms/step - loss: 0.0943 - accuracy: 0.9915 - val_loss: 0.0053 - val_accuracy: 0.9998
Epoch 2/10
3355/3355 [==============================] - 371s 111ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 0.9998
Epoch 3/10
3355/3355 [==============================] - 371s 111ms/step - loss: 8.0134e-04 - accuracy: 1.0000 - val_loss: 9.8978e-04 - val_accuracy: 0.9999
Epoch 4/10
3355/3355 [==============================] - 372s 111ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 9.0866e-04 - val_accuracy: 0.9999
Epoch 5/10
3355/3355 [==============================] - 372s 111ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 9.4450e-04 - val_accuracy: 0.9999
Epoch 6/10
3355/3355 [==============================] - 373s 111ms/step - loss: 2.1540e-04 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 0.9999
Epoch 7/10
3355/3355 [==============================] - 371s 111ms/step - loss: 1.8864e-04 -

In [168]:
y_pred = model.predict(X_test_pad)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

839/839 [==============================] - 22s 25ms/step
Accuracy: 0.9998509687034277
Precision: 0.9999273361430024
Recall: 0.9997820401046208
F1 Score: 0.9998546828453099


In [169]:
import pickle

In [170]:
filename = 'model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(model, file)